Importing necessary libraries

In [44]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Loading the dataset

In [45]:
data = pd.read_csv('/content/tweets.csv')

In [46]:
pd.set_option('display.max_colwidth',None)
data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


Storing stopwords

In [47]:
stopword=nltk.corpus.stopwords.words('english')

Removing stopwords

In [48]:
def sw_removed(text):
  sw =[i for i in text if i not in stopword]
  return sw

In [49]:
import gensim
data['tweet_clear'] = data['tweet'].apply(lambda x:gensim.utils.simple_preprocess(x))

In [50]:
data['tweet_clean']=data['tweet_clear'].apply(lambda x: sw_removed(x))

Viewing the clean tweet corpus after preprocessing

In [51]:
data.head()

,id,label,tweet,tweet_clear,tweet_clean
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone,"[fingerprint, pregnancy, test, https, goo, gl, mfqv, android, apps, beautiful, cute, health, igers, iphoneonly, iphonesia, iphone]","[fingerprint, pregnancy, test, https, goo, gl, mfqv, android, apps, beautiful, cute, health, igers, iphoneonly, iphonesia, iphone]"
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/,"[finally, transparant, silicon, case, thanks, to, my, uncle, yay, sony, xperia, sonyexperias, http, instagram, com, yget, jc, jm]","[finally, transparant, silicon, case, thanks, uncle, yay, sony, xperia, sonyexperias, http, instagram, com, yget, jc, jm]"
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu,"[we, love, this, would, you, go, talk, makememories, unplug, relax, iphone, smartphone, wifi, connect, http, fb, me, lsupcu]","[love, would, go, talk, makememories, unplug, relax, iphone, smartphone, wifi, connect, http, fb, lsupcu]"
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/,"[wired, know, george, was, made, that, way, iphone, cute, daventry, home, http, instagr, am, li_]","[wired, know, george, made, way, iphone, cute, daventry, home, http, instagr, li_]"
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!,"[what, amazing, service, apple, won, even, talk, to, me, about, question, have, unless, pay, them, for, their, stupid, support]","[amazing, service, apple, even, talk, question, unless, pay, stupid, support]"


Spliting train and test datas

In [52]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(data['tweet_clean'],data['label'],test_size =0.3)

Word2Vec model

In [53]:
from gensim.models import Word2Vec
w2v_model = gensim.models.Word2Vec(x_train,min_count = 1)

In [54]:
words = w2v_model.wv.index_to_key

In [55]:
x_train_vec = []
x_test_vec = []

for ls in x_train:
  vec = []
  for word in ls:
    if word in words:
      vec.append(w2v_model.wv['word'])
  x_train_vec.append(np.array(vec))


In [56]:
for ls in x_test:
  vec = []
  for word in ls:
    if word in words:
      vec.append(w2v_model.wv['word'])
  x_test_vec.append(np.array(vec))

Taking average values of datas for easy calculation

In [57]:
x_train_vec_avg = []
x_test_vec_avg = []

def cal_avg_vec(v):
  if v.size:
    return v.mean(axis = 0)
  else:
    np.zeros(100,dtype=float)

for v in x_train_vec:
  x_train_vec_avg.append(cal_avg_vec(v))

for v in x_test_vec:
  x_test_vec_avg.append(cal_avg_vec(v))

In [58]:
x_train_vec_avg = np.array(x_train_vec_avg)
x_test_vec_avg = np.array(x_test_vec_avg)

Training the model using RandomForestClassifier

In [59]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(x_train_vec_avg,y_train)

RandomForestClassifier()

Accuracy Score

In [60]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(x_test_vec_avg)

In [61]:
accuracy_score(y_test,y_pred)

0.7588383838383839